# Lecture 9: 异常 (Exceptions) 和字典 (Dictionaries) 学习笔记

## 第一部分：异常处理 (Exception Handling)

程序在运行时可能会遇到各种错误，比如用户输入了无效数据、尝试除以零、打开一个不存在的文件等。这些在运行时检测到的错误被称为 **异常 (Exceptions)**。

如果异常发生而未被处理，程序会立即崩溃并显示一长串错误信息 (Traceback)。异常处理机制允许我们 "捕获" 这些错误，并执行备用代码，从而让程序更加健壮，不会轻易崩溃。

---

### 1. `try...except` 基本用法

这是最核心的异常处理结构。我们把 **可能出错** 的代码块放在 `try` 里面，然后用 `except` 块来捕获并处理错误。

**例子**: 用户可能会输入非数字字符，或者输入 0 作为除数。

- **`ValueError`**: 当 `int()` 函数无法将输入字符串转换为整数时触发。
- **`ZeroDivisionError`**: 当代码尝试执行除以零的操作时触发。

In [ ]:
# 原始的、脆弱的代码
# 如果输入 "abc" 或者第二个数字输入 0，程序就会崩溃
# a = int(input("Tell me one number: "))
# b = int(input("Tell me another number: "))
# print(a / b)

# 使用 try...except 的健壮代码
try:
    a = int(input("Tell me one number: "))
    b = int(input("Tell me another number: "))
    print(f"The result of a / b is: {a / b}")
# except 后面不指定任何类型，会捕获所有类型的异常
except:
    print("Bug in user input. Please make sure you enter valid numbers and do not divide by zero.")

### 2. 处理特定类型的异常

为了更精确地控制程序的行为，我们可以为不同类型的异常编写不同的 `except` 子句。这允许我们根据具体的错误类型给用户更明确的反馈。

一个 `try` 块后面可以跟多个 `except` 块。

In [ ]:
try:
    a = int(input("Tell me one number: "))
    b = int(input("Tell me another number: "))
    print(f"a / b = {a / b}")
    print(f"a + b = {a + b}")
except ValueError:
    # 这个块只在 int() 转换失败时执行
    print("Could not convert to a number. Please enter integers.")
except ZeroDivisionError:
    # 这个块只在除以零时执行
    print("Can't divide by zero!")
    print("a / b = infinity")
except:
    # 这个块捕获除了上面两种之外的所有其他异常
    print("Something went very wrong.")

### 3. `else` 和 `finally` 子句

`try...except` 结构还有两个可选的子句：

- **`else`**: 如果 `try` 块中的代码 **没有发生任何异常**，`else` 块中的代码就会被执行。这对于将 "成功时才应执行的代码" 与 `try` 块本身分开很有用。
- **`finally`**: 无论 `try` 块中是否发生异常，`finally` 块中的代码 **总会** 被执行。它通常用于执行 "清理" 操作，比如关闭文件或网络连接，确保资源被正确释放。

In [ ]:
try:
    # 尝试打开一个文件并读取内容
    f = open("my_test_file.txt", "w")
    f.write("Hello, world!")
    # 故意制造一个错误来测试 finally
    # x = 1 / 0 
except FileNotFoundError:
    print("Error: The file was not found.")
else:
    print("File written successfully, no exceptions occurred.")
finally:
    # 无论成功还是失败，都确保文件被关闭
    if 'f' in locals() and not f.closed:
        f.close()
        print("File is now closed.")

### 4. 主动抛出异常 (`raise`)

有时，我们需要在自己的代码中主动触发一个异常。例如，当函数的输入参数不满足预设条件时。`raise` 关键字可以让我们做到这一点。

这是一种明确的信号，告诉调用者发生了错误，而不是默默地失败或者返回一个特殊的错误值。

In [ ]:
# 幻灯片中的 sum_digits 例子
def sum_digits(s):
    """
    s is a non-empty string containing digits.
    Returns sum of all chars that are digits.
    """
    # assert 也可以用来检查前置条件，我们稍后会看到
    if not isinstance(s, str):
        raise TypeError("Input must be a string.")
    
    total = 0
    for char in s:
        try:
            val = int(char)
            total += val
        except ValueError:
            # 如果字符串中包含非数字字符，就主动抛出异常
            raise ValueError("Invalid character in string: cannot convert to integer.")
    return total

try:
    print(sum_digits("12345"))  # 正常工作
    print(sum_digits("12a45"))  # 这会触发我们自己 raise 的 ValueError
except ValueError as e:
    print(f"Error caught: {e}")
except TypeError as e:
    print(f"Error caught: {e}")

### 练习 1: `pairwise_div`

**要求**: 编写一个函数，对两个列表进行逐元素相除。如果分母列表 `Ldenom` 中包含 0，则抛出 `ValueError`。

In [ ]:
def pairwise_div(Lnum, Ldenom):
    """
    Lnum and Ldenom are non-empty lists of equal lengths containing numbers.
    Returns a new list whose elements are the pairwise
    division of an element in Lnum by an element in Ldenom.
    Raise a ValueError if Ldenom contains 0.
    """
    result = []
    for i in range(len(Ldenom)):
        if Ldenom[i] == 0:
            raise ValueError("Denominator list cannot contain zero.")
        result.append(Lnum[i] / Ldenom[i])
    return result

# 示例
L1 = [4, 5, 6]
L2 = [1, 2, 3]
print(f"L1/L2 = {pairwise_div(L1, L2)}")

L3 = [1, 0, 3]
try:
    print(pairwise_div(L1, L3))
except ValueError as e:
    print(f"Caught expected error: {e}")

---
## 第二部分：断言 (Assertions)

**断言 (`assert`)** 是一种用于调试的工具，它可以帮助程序员在代码中设置检查点，确保程序的某个条件必须为真。如果 `assert` 后面的条件为 `False`，程序会立即停止并抛出 `AssertionError`。

> **核心区别**:
> - **异常 (Exceptions)** 用于处理 **可能发生** 的运行时错误（如用户输入错误），它们是程序正常流程的一部分，应该被 `try...except` 捕获和处理。
> - **断言 (Assertions)** 用于检查 **绝不应该发生** 的情况。它是在声明一个程序员坚信为真的事实。如果断言失败，说明程序有 bug，需要程序员去修复，而不是让程序在运行时去处理。

In [ ]:
def calculate_average(grades):
    # 我们断言 grades 列表不应该是空的，因为计算一个空列表的平均分是没有意义的
    # 这是一个内部逻辑检查，而不是为了处理用户输入
    assert len(grades) != 0, "List of grades cannot be empty."
    return sum(grades) / len(grades)

# 正常调用
print(calculate_average([80, 90, 100]))

# 这个调用会触发 AssertionError，因为它违反了函数内部的假设
try:
    print(calculate_awesome([]))
except AssertionError as e:
    print(f"Assertion failed: {e}")

## 第三部分：字典 (Dictionaries)

如果说 `list` 是通过 **整数索引** (0, 1, 2...) 来访问元素的有序集合，那么 **字典 (`dict`)** 就是通过 **键 (key)** 来访问 **值 (value)** 的无序集合。

字典非常适合用来存储具有映射关系的数据，例如：姓名 -> 电话号码，单词 -> 释义，学生ID -> 学生信息。

### 1. 字典的创建与访问

- 字典用花括号 `{}` 创建。
- 每个元素是一个 `key: value` 对。
- `key` 必须是 **不可变** 类型（如字符串、数字、元组），且必须是唯一的。
- `value` 可以是任何类型。

In [ ]:
# 创建一个字典
grades = {'Ana': 'B', 'Matt': 'A', 'John': 'B', 'Katy': 'A'}
print(f"Original grades: {grades}")

# 访问一个值
johns_grade = grades['John']
print(f"John's grade is: {johns_grade}")

# 尝试访问一个不存在的键会引发 KeyError
try:
    grace_grade = grades['Grace']
except KeyError:
    print("Key 'Grace' not found in the dictionary.")

### 2. 字典的操作

字典是 **可变** 的，我们可以随时添加、修改或删除其中的元素。

- **添加/修改**: `my_dict[new_key] = new_value`
- **删除**: `del my_dict[key]`
- **检查键是否存在**: `key in my_dict`

In [ ]:
grades = {'Ana': 'B', 'Matt': 'A', 'John': 'B', 'Katy': 'A'}

# 添加一个新条目
grades['Grace'] = 'A'
print(f"After adding Grace: {grades}")

# 修改一个现有条目
grades['Matt'] = 'C'
print(f"After changing Matt's grade: {grades}")

# 删除一个条目
del grades['Ana']
print(f"After deleting Ana: {grades}")

# 检查一个键是否存在
print(f"Is 'John' in grades? {'John' in grades}")
print(f"Is 'Ana' in grades? {'Ana' in grades}")

# 注意：'in' 关键字只检查键，不检查值
print(f"Is 'A' in grades? {'A' in grades}") # This will be False

### 3. 遍历字典

有三种主要的方式来遍历字典：

1.  **遍历键 (`.keys()`)**:
2.  **遍历值 (`.values()`)**:
3.  **遍历键值对 (`.items()`)**: 这是最常用和最方便的方式。```

```python
grades = {'Ana': 'B', 'Matt': 'A', 'John': 'B', 'Katy': 'A'}

print("\n--- Iterating over keys ---")
for student in grades.keys():
    print(student)

print("\n--- Iterating over values ---")
for grade in grades.values():
    print(grade)

print("\n--- Iterating over items (key-value pairs) ---")
for student, grade in grades.items():
    print(f"Key: {student} has Value: {grade}")

### 核心知识点解析：为什么字典的键必须是不可变的？

这与字典的内部实现方式——**哈希表 (Hash Table)** 有关。

1.  **哈希函数**: 当你向字典中添加一个键值对时，Python 会对 **键 (key)** 运行一个叫做 "哈希函数" 的算法，这个函数会把键转换成一个唯一的整数（哈希值）。
2.  **内存地址**: 这个哈希值被用作一个索引，直接指向内存中的一个位置，**值 (value)** 就存储在这个位置。
3.  **快速查找**: 当你查询一个键时，Python 再次对键运行同样的哈希函数，得到相同的哈希值，然后直接去内存中对应的位置取出值。这个过程非常快，时间复杂度接近 O(1)，远快于在列表中搜索元素。

**关键在于**: 如果键是可变的（比如一个列表），那么它的内容就可能改变。一旦内容改变，它的哈希值也会随之改变。这样一来，当你再去查找这个键时，会得到一个新的哈希值，就再也找不到原来存储数据的位置了。

因此，为了保证哈希值的一致性和查找的可靠性，字典的键必须是 **不可变 (immutable)** 的。

![Hash Function Visualization](https://i.imgur.com/5E6Qj5B.png)
*(图片引用自幻灯片第37页)*

---
## 第四部分：综合实例 - 歌词词频统计

这是一个非常经典和实用的例子，它完美地展示了字典的强大之处。

**目标**:
1.  创建一个频率字典，映射 `单词 -> 出现次数`。
2.  找出出现次数最多的单词。
3.  找出所有出现次数超过 `x` 次的单词，并按频率排序。

### 步骤 1: 生成频率字典

In [ ]:
song = "RAH RAH AH AH AH ROM MAH RO MAH MAH"

def generate_word_dict(song_text):
    """从一个字符串生成词频字典"""
    word_dict = {}
    # 统一转换为小写，并按空格分割成单词列表
    words_list = song_text.lower().split()
    
    for word in words_list:
        if word in word_dict:
            # 如果单词已经在字典里，计数加一
            word_dict[word] += 1
        else:
            # 如果是第一次见到这个单词，创建新条目，计数为 1
            word_dict[word] = 1
            
    return word_dict

word_freq = generate_word_dict(song)
print(f"Frequency Dictionary: {word_freq}")

### 步骤 2: 找到频率最高的单词

In [ ]:
def find_frequent_word(word_dict):
    """找到字典中值最大的键值对(们)"""
    if not word_dict:
        return ([], 0)
        
    # 首先找到最高频率（最大的 value）
    highest_freq = max(word_dict.values())
    
    # 然后找到所有频率等于最高频率的单词
    words = []
    for word, freq in word_dict.items():
        if freq == highest_freq:
            words.append(word)
            
    return (words, highest_freq)

most_common = find_frequent_word(word_freq)
print(f"Most common words: {most_common[0]} with frequency {most_common[1]}")

### 步骤 3: 找到所有频率高于 `x` 的单词

这是一个巧妙的算法，它通过 **循环和修改字典** 来实现：

1.  在一个 `while` 循环中，不断找出当前字典中频率最高的单词。
2.  如果这个最高频率大于 `x`，就将其添加到结果列表中。
3.  **关键**: 从字典中删除这些刚刚找到的单词。
4.  这样，下一次循环调用 `find_frequent_word` 时，找到的就是 "第二高" 频率的单词了。
5.  循环直到最高频率不再大于 `x` 为止。

In [ ]:
def occurs_often(original_word_dict, x):
    """
    找到所有出现次数 > x 的单词，并按频率从高到低排序。
    """
    # 创建一个副本，以防修改原始字典
    word_dict = original_word_dict.copy()
    
    result_list = []
    
    while True:
        # 找到当前字典中频率最高的单词
        frequent_tuple = find_frequent_word(word_dict)
        words, freq = frequent_tuple
        
        if freq > x:
            # 如果频率符合要求，添加到结果中
            result_list.append(frequent_tuple)
            # 从字典中删除这些单词，以便下次循环
            for word in words:
                del word_dict[word]
        else:
            # 如果最高频率都不满足要求了，就可以停止循环
            break
            
    return result_list

# 找到所有出现次数 > 1 的单词
frequent_words_list = occurs_often(word_freq, 1)
print(f"Words with frequency > 1: {frequent_words_list}")

---
### 总结

- **异常处理 (`try...except`)** 是编写健壮程序的关键，它能优雅地处理运行时错误。
- **断言 (`assert`)** 是程序员的调试工具，用于检查代码中必须为真的内部逻辑条件。
- **字典 (`dict`)** 是一种极其强大的数据结构，用于存储键值对，并提供飞快的查找速度。它是解决许多编程问题的利器。